In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Anorexia_Nervosa"
cohort = "GSE60190"

# Input paths
in_trait_dir = "../../input/GEO/Anorexia_Nervosa"
in_cohort_dir = "../../input/GEO/Anorexia_Nervosa/GSE60190"

# Output paths
out_data_file = "../../output/preprocess/Anorexia_Nervosa/GSE60190.csv"
out_gene_data_file = "../../output/preprocess/Anorexia_Nervosa/gene_data/GSE60190.csv"
out_clinical_data_file = "../../output/preprocess/Anorexia_Nervosa/clinical_data/GSE60190.csv"
json_path = "../../output/preprocess/Anorexia_Nervosa/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genetic Neuropathology of Obsessive Psychiatric Syndromes"
!Series_summary	"Anorexia nervosa (AN), bulimia nervosa (BN), and obsessive-compulsive disorder (OCD) are complex psychiatric disorders with shared obsessive features, thought to arise from the interaction of multiple genes of small effect with environmental factors.  Potential candidate genes for AN, BN, and OCD have been identified through clinical association and neuroimaging studies; however, recent genome-wide association studies of eating disorders (ED) so far have failed to report significant findings. Additionally, few if any studies have interrogated postmortem brain tissue for evidence of eQTLs associated with candidate genes, which has particular promise as an approach to elucidating molecular mechanisms of association. We therefore selected single nucleotide polymorphisms (SNPs) based on candidate gene studies for AN, BN, and OCD from the literature, and examined the associatio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from DLPFC tissue
# run on the Illumina HumanHT-12 v3 microarray, which is gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Trait Data (Anorexia Nervosa)
# Looking at the sample characteristics, we can see that row 3 contains diagnoses information
# and row 1 contains OCD status which includes "ED" (eating disorder)
# Since our trait is Anorexia_Nervosa, and the dataset mentions ED (eating disorders)
# we'll use row 3 (dx field) which has more specific diagnostic categories
trait_row = 3

# 2.2 Age Data
# Row 5 contains age information
age_row = 5

# 2.3 Gender Data
# Row 7 contains Sex information
gender_row = 7

# 3. Define conversion functions

def convert_trait(value):
    """Convert trait value to binary format (0=Control, 1=Anorexia_Nervosa)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates Anorexia Nervosa
    # From the sample characteristics, 'ED' refers to eating disorder patients
    if value == 'ED':
        return 1
    elif value == 'Control':
        return 0
    else:
        # Other diagnoses are not our target trait
        return 0

def convert_age(value):
    """Convert age value to continuous format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary format (0=Female, 1=Male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'F':
        return 0
    elif value == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical features dataframe
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features as CSV
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM1467273': [0.0, 50.421917, 0.0], 'GSM1467274': [0.0, 27.49863, 1.0], 'GSM1467275': [0.0, 30.627397, 1.0], 'GSM1467276': [0.0, 61.167123, 1.0], 'GSM1467277': [0.0, 32.69589, 1.0], 'GSM1467278': [0.0, 39.213698, 0.0], 'GSM1467279': [0.0, 58.605479, 1.0], 'GSM1467280': [0.0, 49.2, 1.0], 'GSM1467281': [0.0, 41.041095, 1.0], 'GSM1467282': [0.0, 51.750684, 1.0], 'GSM1467283': [0.0, 50.89863, 1.0], 'GSM1467284': [0.0, 26.745205, 1.0], 'GSM1467285': [0.0, 29.104109, 1.0], 'GSM1467286': [0.0, 39.301369, 0.0], 'GSM1467287': [0.0, 48.978082, 1.0], 'GSM1467288': [0.0, 57.884931, 1.0], 'GSM1467289': [0.0, 28.364383, 1.0], 'GSM1467290': [0.0, 24.041095, 1.0], 'GSM1467291': [0.0, 19.268493, 0.0], 'GSM1467292': [0.0, 27.230136, 0.0], 'GSM1467293': [0.0, 46.605479, 1.0], 'GSM1467294': [0.0, 23.443835, 0.0], 'GSM1467295': [0.0, 51.038356, 1.0], 'GSM1467296': [0.0, 39.663013, 1.0], 'GSM1467297': [0.0, 46.109589, 1.0], 'GSM1467298': [0.0, 77.989041, 1.0], 'GSM1467299': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Anorexia_Nervosa/GSE60190/GSE60190_series_matrix.txt.gz


Gene data shape: (48679, 133)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed in the gene expression data, 
# these are Illumina probe IDs (starting with ILMN_) rather than standard human gene symbols.
# Illumina probe IDs need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: Illumina HumanHT-12 V3.0 expression beadchip



Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966', 'ILMN_1668162', 'ILMN_1715600', 'ILMN_1912287', 'ILMN_1793729', 'ILMN_1889125'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ', 'rtCnUep15THUpc_0e4', 'QtVBXBWhekTEIT0kjo', 'EuUnlPkeXRP9fyO.iQ', '0flyIEROp.olYSF6n4', 'fegCQD_j_69DUU38dI'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'Unigene', 'RefSeq', 'Unigene'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756', 'ILMN_7652', 'ILMN_35097', 'ILMN_77451', 'ILMN_18382', 'ILMN_108888'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756', 'ILMN_7652', 'ILMN_35097', 'ILMN_77451', 'IL

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers and determine the appropriate columns
# From the preview, we can see the gene expression data uses 'ILMN_XXXXX' identifiers (Illumina probe IDs)
# In the annotation dataframe, the 'ID' column contains these same Illumina probe IDs
# The 'Symbol' column contains the gene symbols we need to map to

# 2. Get a gene mapping dataframe by extracting the relevant columns
# Use the get_gene_mapping function to create a mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 10 rows of gene mapping:")
print(gene_mapping.head(10))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles probes mapping to multiple genes and genes with multiple probes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to ensure consistency across datasets
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalizing symbols: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (36157, 2)
First 10 rows of gene mapping:
              ID       Gene
0   ILMN_1725881   LOC23117
2   ILMN_1804174     FCGR2B
3   ILMN_1796063     TRIM44
4   ILMN_1811966  LOC653895
5   ILMN_1668162    DGAT2L3
6   ILMN_1715600  LOC387701
8   ILMN_1793729   C15orf39
10  ILMN_2296644    PCDHGA9
11  ILMN_1711283    PCDHGA9
12  ILMN_1682799   STAMBPL1
Gene expression data shape after mapping: (19091, 133)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M',
       'A2ML1', 'A3GALT2'],
      dtype='object', name='Gene')


Gene expression data shape after normalizing symbols: (18298, 133)
First 10 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Anorexia_Nervosa/gene_data/GSE60190.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - we already did this in step 6
# No need to do it again
print(f"Gene data already normalized and saved to {out_gene_data_file}")

# 2. Load clinical data that was already saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_data_df.shape}")
print(clinical_data_df.head())

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from dorsolateral prefrontal cortex of postmortem tissue with Eating Disorders."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data already normalized and saved to ../../output/preprocess/Anorexia_Nervosa/gene_data/GSE60190.csv
Loaded clinical data shape: (3, 133)
                  GSM1467273  GSM1467274  GSM1467275  GSM1467276  GSM1467277  \
Anorexia_Nervosa    0.000000     0.00000    0.000000    0.000000     0.00000   
Age                50.421917    27.49863   30.627397   61.167123    32.69589   
Gender              0.000000     1.00000    1.000000    1.000000     1.00000   

                  GSM1467278  GSM1467279  GSM1467280  GSM1467281  GSM1467282  \
Anorexia_Nervosa    0.000000    0.000000         0.0    0.000000    0.000000   
Age                39.213698   58.605479        49.2   41.041095   51.750684   
Gender              0.000000    1.000000         1.0    1.000000    1.000000   

                  ...  GSM1467396  GSM1467397  GSM1467398  GSM1467399  \
Anorexia_Nervosa  ...    1.000000    0.000000    0.000000    0.000000   
Age               ...   30.726027   63.471232   54.808219   57.512328

Data shape after handling missing values: (133, 18301)
For the feature 'Anorexia_Nervosa', the least common label is '1.0' with 3 occurrences. This represents 2.26% of the dataset.
The distribution of the feature 'Anorexia_Nervosa' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 30.043835
  50% (Median): 45.224657
  75%: 54.424657
Min: 16.183561
Max: 84.057534
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 36 occurrences. This represents 27.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Anorexia_Nervosa/cohort_info.json
Dataset is not usable for analysis. No linked data file saved.
